In [66]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

#get the data from link and process it in BeautifulSoup and read the data in csv

In [67]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(link).text

soup = BeautifulSoup(source, 'lxml')
csv_file = open('postal_code.csv', 'w')

csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood'])

tables = soup.find("table", class_="wikitable sortable")


In [68]:
from collections import defaultdict

#group data of same key in a single row 

In [69]:
grouped_data = defaultdict(list)

for tr in tables.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    
    key = tds[0].text
    grouped_data[key].append([tds[1].text, tds[2].text.strip('\n')])



#clean the data according to the constrain define in problem

In [70]:
naVal = False
cleaned_data = defaultdict(list)
for key, vals in grouped_data.items():
    for val in vals:
        if val[0] == 'Not assigned':
            naVal = True
        elif val[1] == 'Not assigned':
            val[1] = val[0]
            naVal = False
        else:
            naVal = False
    
    if naVal:
        continue
    else:
        cleaned_data[key].append(vals)


#write the cleaned data in csv file

In [71]:
borough = ''
for key, values in cleaned_data.items():
    multi_neigh = ''
    for vals in values:
        for val in vals:
            multi_neigh += val[1] + ','
            borough = val[0]
    csv_writer.writerow([key, borough, multi_neigh.strip(',')])

csv_file.close()

#Open file in pandas data frame

In [72]:
df = pd.read_csv("postal_code.csv")
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [73]:
df.shape

(103, 3)